# ML в Биологии
## Features Importance

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

import seaborn as sns

sns.set(context='poster')
%matplotlib inline

### Задача 1

Рассмотрим как можно провести отбор признаков с помощью обучения линейной регрессии и леса с малым количеством деревьев ($\approx 10$) на примере задачи регрессии. Будем использовать датасет <https://www.kaggle.com/datasets/abrambeyer/openintro-possum> и пытаться предсказать возраст оппосумов на основе различных параметров их тела.

Для упрощения задачи избавимся от категориальных признаков

In [ ]:
!unzip archive

In [ ]:
data = pd.read_csv('possum.csv').dropna()
X = data[data.columns.drop(['sex', 'Pop', 'age'])]
y = data['age']

Разделим выборку на тренировочную и тестовую часть

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные для корректной работы **линейной регрессии**:

In [ ]:
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

Обучим модель линейной регрессии с L1-регуляризацией на всех данных, посмотрим на метрику качества

In [ ]:
lin_model = Lasso(alpha=0.05, fit_intercept=True).fit(X_train_norm, y_train)
print('Коэффициенты модели:', lin_model.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model.predict(X_test_norm), y_test))

Проделаем аналогичную процедуру с лесом c небольшим количеством деревьев:

In [ ]:
forest = RandomForestRegressor().fit(X_train_norm, y_train)
print('Важность признаков:', forest.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest.predict(X_train), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest.predict(X_test), y_test))

Что вы можете сказать смотря на коэффициенты модели для регрессии и на важность признаков для леса?

**Вывод**:

Видно, что не все параметры для важны. Некоторые из них оказывают большее влияние. Такая тенденция видна и в лесу, и в регрессии. Чем меньше параметр - тем меньше его важность.

Составим 2 списка наиболее важных признаков - один с признаками, наиболее важными для линейной регрессии, второй с наиболее важными для леса

In [ ]:
lin_imp = []
forest_imp = []
for i, column in enumerate(X.columns):
    if lin_model.coef_[i] > 0.1:
        lin_imp.append(i)
    if forest.feature_importances_[i] > 0.1:
        forest_imp.append(column)

In [ ]:
X_train_norm_lin_imp = X_train_norm[:, lin_imp]
X_test_norm_lin_imp = X_test_norm[:, lin_imp]

X_train_forest_imp = X_train[forest_imp]
X_test_forest_imp = X_test[forest_imp]

Теперь обучим модели только на важных признаках:

In [ ]:
lin_model_2 = Lasso(alpha=0.01, fit_intercept=True).fit(X_train_norm_lin_imp, y_train)
print('Коэффиценты модели:', lin_model.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model_2.predict(X_train_norm_lin_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model_2.predict(X_test_norm_lin_imp), y_test))

In [ ]:
forest_2 = RandomForestRegressor().fit(X_train_forest_imp, y_train)
print('Важность признаков:', forest.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest_2.predict(X_train_forest_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest_2.predict(X_test_forest_imp), y_test))

Что вы можете сказать о качестве предсказания?

**Вывод:**

В линейной регрессии не сильно изменилась метрика. С помощью L1-регуляризации мы отбираем важные признаки. А вот лес стал гораздо лучше предсказывать.

Рассмотрим работу других метотодов оценки важности признаков, а именно *Permutation feature importance* и *Column feature importance*,на примере [KNN-регресии](https://scikit-learn.org/1.5/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

Для корректной работы KNN необходимо стандартизовать признаки, как мы это делали для Lasso-регресии. А также разделить тренировачный датасет на train и val.

In [ ]:
X_train_norm, X_val_norm, y_train, y_val = train_test_split(X_train_norm, y_train, test_size=0.33, random_state=42)

In [ ]:
knn_1 = KNeighborsRegressor(n_neighbors=5).fit(X_train_norm, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn_1.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn_1.predict(X_test_norm), y_test))

**Permutation feature importance** полностью реализован в `sklearn.inspection`


Функция `permutation_importance()` принимает на вход:
- `model` &mdash; обученная модель
- `X, y` &mdash;  фичи и таргет валидационной части датасета
- `n_repeats` &mdash; сколько раз переставляется фича

На выходе мы получаем:
- `importances` &mdash сырые оценки значимости для всех фичей и всех итераций
- `importances_mean` &mdash; среднее по всем итерациям
- `importances_std` &mdash; стандартоное отклонение среднего

Оценим важность признаков

In [ ]:
r = permutation_importance(knn_1, X_val_norm, y_val, n_repeats=5)

Отберем признаки согласно нашей оценке

In [ ]:
knn_imp = []
for i, column in enumerate(X.columns):
    if r.importances_mean[i] - r.importances_std[i] >= 0:
        knn_imp.append(i)

In [ ]:
X_train_norm_knn_imp = X_train_norm[:, knn_imp]
X_test_norm_knn_imp = X_test_norm[:, knn_imp]

In [ ]:
knn_2 = KNeighborsRegressor(n_neighbors=5).fit(X_train_norm_knn_imp, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn_2.predict(X_train_norm_knn_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn_2.predict(X_test_norm_knn_imp), y_test))

**Drop-Column feature importance**

Для него готовой реализации в `sklearn` нет, так что воспольлзуемся кодом ниже.

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5).fit(X_train_norm, y_train)
baseline = MSE(knn.predict(X_val_norm), y_val)
knn_imp = []

for i, column in enumerate(X.columns):
    X_train_drop = np.delete(X_train_norm, i, 1)
    X_val_drop =  np.delete(X_val_norm, i , 1)

    knn_drop = KNeighborsRegressor(n_neighbors=5).fit(X_train_drop, y_train)

    mse = MSE(knn_drop.predict(X_val_drop), y_val)

    if ((mse - baseline) / baseline) > 0.1:
        knn_imp.append(i)

In [ ]:
X_train_norm_knn_imp = X_train_norm[:, knn_imp]
X_test_norm_knn_imp = X_test_norm[:, knn_imp]

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5).fit(X_train_norm_knn_imp, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn.predict(X_train_norm_knn_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn.predict(X_test_norm_knn_imp), y_test))

**Вывод:**

После permutation метрика на KNN ухудшается,а на Drop Column метрика меняется незначительно. Вероятно, что удаляется слишком много признаков и это как-то можно исправить более тщательным анализом и настройкой фичей.